This notebook follows the plan:
- Import the modules
- Import the "basic" data (movies and characters datasets from CMU), clean it and save it
- Extraction of the lemmatized version of the plot summaries from the corenlp processed data
- Processing of the summaries according to the gender
- Loading, cleaning of IMDb dataset
- Matching CMU and IMDb datasets

# Import the modules

In [1]:
import pandas as pd
import numpy as np
import pickle

# Import the data

In [2]:
# File and folder names
DATA_FOLDER = 'Data/'
CHARACTER_DATASET = DATA_FOLDER + 'character.metadata.tsv'
MOVIE_DATASET = DATA_FOLDER + 'movie.metadata.tsv'

SUMMARIES_DATASET = DATA_FOLDER + 'plot_summaries.txt'
NLP_FOLDER = DATA_FOLDER + 'corenlp_plot_summaries/'
DEFAULT_COMPRESSION = 'gzip'

In [3]:
# Function to load data
def load_metadata(path, column_names, header=None, low_memory=False):
    return pd.read_table(path, header=header, names=column_names)

In [5]:
# Name columns
columns_character = ['Wikipedia_movie_ID', 'Freebase_movie_ID', 'Movie_release_date', 'Character_name', 'Actor_date_of_birth', 'Actor_gender', 'Actor_height_meters', 'Actor_ethnicity_Freebase_ID', 'Actor_name', 'Actor_age_at_movie_release', 'Freebase_character_actor_map_ID', 'Freebase_character_ID', 'Freebase_actor_ID']
columns_movie = ['Wikipedia_movie_ID', 'Freebase_movie_ID', 'Movie_name','Movie_release_date','Movie_box_office_revenue', 'Movie_runtime','Movie_languages','Movie_countries','Movie_genres' ]

# Load data with correct column names
characters = load_metadata(CHARACTER_DATASET,column_names=columns_character)
movies = load_metadata(MOVIE_DATASET,column_names=columns_movie)

In [6]:
# Load summaries
with open(SUMMARIES_DATASET,'r', encoding='utf-8') as file:
    summaries = file.readlines()

## First glimpse at the data

First we observe the movies dataframe:

In [7]:
print(len(movies))
movies.head(2)

81741


,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."


Then we observe the characters dataframe:

In [8]:
print(len(characters))
characters.head(2)

450669


,Wikipedia_movie_ID,Freebase_movie_ID,Movie_release_date,Character_name,Actor_date_of_birth,Actor_gender,Actor_height_meters,Actor_ethnicity_Freebase_ID,Actor_name,Actor_age_at_movie_release,Freebase_character_actor_map_ID,Freebase_character_ID,Freebase_actor_ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.62,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.78,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4


We also check the summaries:

In [9]:
print('Number of plots:', len(summaries))
summaries[0]

Number of plots: 42306


"23890098\tShlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.\n"

# Cleaning

## Problem of dates

We fix typos and absurd dates

In [10]:
movies.loc[movies.Movie_release_date == '1010-12-02','Movie_release_date'] = '2010-12-02'
characters.loc[characters.Movie_release_date == '1010-12-02','Movie_release_date'] = '2010-12-02'
characters[characters.Actor_date_of_birth == '2050'] = '1971'
characters = characters.drop(characters[characters.Actor_date_of_birth < '1500'].index)
characters = characters.drop(characters[characters.Actor_date_of_birth > '2030'].index)

## Format of movie languages, genres and country

Convert the format of languages, genres, country columns to a simpler format (in terms of utilisation).

In [11]:
def format_multiple(chain,deb,step):
    '''Split the chain of characters at each " encountered, and keep only the element in deb +i*step'''
    res = chain.split('"')[deb::step]
    return res

In [12]:
movies.loc[:,'Movie_genres'] = movies.Movie_genres.apply(format_multiple,deb=3,step=4)
movies.loc[:,'Movie_countries'] = movies.Movie_countries.apply(format_multiple,deb=3,step=4)
movies.loc[:,'Movie_languages'] = movies.Movie_languages.apply(format_multiple,deb=3,step=4)

In [13]:
keys = ['Movie_languages','Movie_countries','Movie_genres']
for key in keys:
    nb = len(movies[movies[key].apply(len) == 0])
    print('{nb} movies without {key} ({percentage:.2f}% of the dataset)'.format(nb=nb,key=key, percentage=nb*100/len(movies)))

13866 movies without Movie_languages (16.96% of the dataset)
8154 movies without Movie_countries (9.98% of the dataset)
2294 movies without Movie_genres (2.81% of the dataset)


## Format for dates

For our study, we only keep the years from the dates.

In [14]:
movies.Movie_release_date = pd.to_datetime(movies.Movie_release_date,format='%Y-%m-%d').dt.year
characters.Movie_release_date = pd.to_datetime(characters.Movie_release_date,format='%Y-%m-%d').dt.year
characters.Actor_date_of_birth = pd.to_datetime(characters.Actor_date_of_birth,format='%Y-%m-%d',utc=True,errors='coerce').dt.year

## Saving the new dataset

We pickle our data in order to reuse directly the cleaned data (and load it faster).

In [15]:
DESTINATION = './Data/'
EXT = '.pkl'
to_pickle_data = [characters,movies]
to_pickle_name = ['characters','movies']
for i in range(len(to_pickle_data)):
    to_pickle_data[i].to_pickle(DESTINATION+to_pickle_name[i]+EXT)

# # To unpickle:
# characters = pd.read_pickle("./Data/characters.pkl") 
# movies = pd.read_pickle("./Data/movies.pkl")

# Lemmatizing the summaries

We lemmatize data (for examples *'is'* becomes *'be'*) to be able to count words better. To do so, we used the `corenlp_plot_summaries` files, and exctracted from it the lemmatized versions of the movies summaries.

In [45]:
# Set to True to save the data
LEMMATIZE_SUMMARIES = False # Takes ~7 mins to run (on i7-10875H CPU)

if LEMMATIZE_SUMMARIES:
    # Imports
    from time import time
    import os
    import gzip
    import re

    # Count the number of files in the directory
    nb_files = 0
    for filename in os.listdir(NLP_FOLDER):
        path = os.path.join(NLP_FOLDER, filename)
        nb_files += 1
    print('Number of summaries:',nb_files)

    ext = '.xml.gz' # Extension name
    dico_processed_summmaries = {} # Dictionary to store the processed summaries
    regex = r'<lemma>.*?</lemma>' # Expression to detect in the corenlp data <lemma>(word)</lemma>

    deb = time() # Start timer
    count = 0 # Counter

    # Iteration over the files
    for filename in os.listdir(NLP_FOLDER):
        path = os.path.join(NLP_FOLDER, filename) # Path to the file
        id_summary = path[len(NLP_FOLDER):-len(ext)] # id of the summary = filename without extension
        summary = '' # String to store the summary

        if os.path.isfile(path): # Checking if it is a file
            with gzip.open(path, 'rb') as f: # Opening the .gz file
                for line in f:
                    txt = line.decode().strip() # Extract the line as txt
                    for elt in re.finditer(regex,txt): # Find all the elements like regex
                        summary += re.split('[><]',elt.group(0))[2].lower() + ' ' # Adding only the lemmatized word
        
        # Set the summary in the dictionary and increment the counter
        dico_processed_summmaries[id_summary] = summary
        count += 1

        # Evolution of the process
        if count%1000 == 0:
            print('{processed}/{tot} files processed --> {perc:.1f}% ({t:.1f} seconds since deb)'.format(processed=count,tot=nb_files,perc=count/nb_files*100,t=time()-deb))
    
    # Pickle the file
    with open(DATA_FOLDER + 'nlp_summaries.pkl', 'wb') as file:
        pickle.dump(dico_processed_summmaries, file, protocol=pickle.HIGHEST_PROTOCOL)

Let us try to extract the data:

In [46]:
# Read the pickle file
nlp_summaries = pd.read_pickle(DATA_FOLDER+'nlp_summaries.pkl')

# Observe the first lemmatized summary
for key,value in nlp_summaries.items():
    print('Key:',key)
    print('Summary:\n',value[:200]+'...')
    break

Key: 236497
Summary:
 king arthur be sit with he knight of the round table , complain about hard time that have befall the kingdom ever since the black knight steal the singing sword . he ask he knight - among they sir osi...


# Separating sentences between sexes

The aim of this part is to separate sentences between sexes to do a sentimental analysis later. To do so, we check if a feminine actor or the *'she'* pronoun is present in a sentence and add them to a new file. We do the same for a male actor and the *'he'* pronoun. Note that for example the sentence *'She hates him'* will become *'she hate he'* once lemmatized, which will be put in the feminine and maculine files

This approach is not perfect, since for example in the sentences 'She likes butter. Indeed, the actress loves food.', only the first one will be added. It is not perfect, but the best solution we could think of.

In [52]:
# Create a dataframe with the characters
characters_per_film = characters.copy()
# Put the column in their correct type and lower chars
characters_per_film['Wikipedia_movie_ID'] = characters_per_film['Wikipedia_movie_ID'].astype(int)
characters_per_film['Character_name'] = characters_per_film['Character_name'].astype(str).apply(lambda x: x.lower())
# Sort the dataframe by movie ID
characters_per_film = characters_per_film.sort_values(by=['Wikipedia_movie_ID'])
# Drio rows where the character name or the gender is empty
characters_per_film = characters_per_film.dropna(subset=['Character_name', 'Actor_gender'])
# Group the dataframe by movie ID
characters_per_film = characters_per_film.groupby('Wikipedia_movie_ID')[['Wikipedia_movie_ID', 'Character_name', 'Actor_gender']]

In [53]:
# Import dataframe from lemmatized summaries
df = pd.DataFrame(list(nlp_summaries.items()), columns = ['id','plot_lemmatized'])
# Put column in their correct type
df['id'] = df['id'].astype(int)
# Sort the dataframe by movie ID
df = df.sort_values(by=['id'])
# Show the first 5 rows
df.head()

,id,plot_lemmatized
12205,330,in order to prepare the role of a important ol...
23703,3217,"after be pull through a time portal , ash will..."
36696,3333,the film follow two juxtapose family : the nor...
40296,3746,-lcb- -lcb- hatnote -rcb- -rcb- in los angeles...
25811,3837,"in the american old west of 1874 , constructio..."


In [54]:
# Set to True to save the data
SEPARATE_SENTENCES = False # Takes ~20 mins to run (on i7-10875H CPU)

if SEPARATE_SENTENCES:
    # Imports
    count = 0
    dico_male = {}
    dico_female = {}

    # Loop on subgroups
    for _, group in characters_per_film:
        # Get the movie id
        movie_id = group['Wikipedia_movie_ID'].iloc[0]
        female_sentences = []
        male_sentences = []

        # Check if wikipedia movie id is in the nlp summaries
        if movie_id in df['id'].values:
            index = df[df['id'] == movie_id].index[0] # Take the correct index
            plot = df['plot_lemmatized'][index] # Take the correct plot
            sentences = plot.split('.') # Split into sentences
            # Loop on sentences
            for sentence in sentences:
                # Loop on characters
                for character in group['Character_name']:
                    # Find the sex of the character
                    gender = group[group['Character_name'] == character].Actor_gender.values[0]
                    # Find potential pronouns discriminative on gender
                    he_index = sentence.find('he')
                    she_index = sentence.find('she')
                    # Check if the pronoun or actor name is in the sentence
                    if ((character in sentence) or (she_index >= 0) or (he_index >= 0)):
                        # Store in dictionary depending on gender of sentence (can also be in both)
                        if ((gender== '1') or (she_index >= 0)):
                            female_sentences.append(sentence)
                        if ((gender == '0') or (he_index >= 0)):
                            male_sentences.append(sentence)

        # Store in dictionary and increment counter
        dico_male[movie_id] = male_sentences
        dico_female[movie_id] = female_sentences
        count += 1

        # Evolution of the process
        if count%1000 == 0:
            print('{processed} files processed'.format(processed=count))

    # Pickle the file
    with open(DATA_FOLDER + 'male_sentences.pkl', 'wb') as file:
        pickle.dump(dico_male, file, protocol=pickle.HIGHEST_PROTOCOL)    
    with open(DATA_FOLDER + 'female_sentences.pkl', 'wb') as file:
        pickle.dump(dico_female, file, protocol=pickle.HIGHEST_PROTOCOL)    

1000 files processed
2000 files processed
3000 files processed
4000 files processed
5000 files processed
6000 files processed
7000 files processed
8000 files processed
9000 files processed
10000 files processed
11000 files processed
12000 files processed
13000 files processed
14000 files processed
15000 files processed
16000 files processed
17000 files processed
18000 files processed
19000 files processed
20000 files processed
21000 files processed
22000 files processed
23000 files processed
24000 files processed
25000 files processed
26000 files processed
27000 files processed
28000 files processed
29000 files processed
30000 files processed
31000 files processed
32000 files processed
33000 files processed
34000 files processed
35000 files processed
36000 files processed
37000 files processed
38000 files processed
39000 files processed
40000 files processed
41000 files processed
42000 files processed
43000 files processed
44000 files processed
45000 files processed
46000 files process

# Enriching the CMU dataset with IMDb dataset

## Loading the data and first glimpse

In [16]:
#Load the most useful datasets for the moment
TITLE_BASICS_DATASET = DATA_FOLDER + 'title.basics.tsv.gz'
TITLE_RATINGS_DATASET = DATA_FOLDER + 'title.ratings.tsv.gz'

columns_title_basics = ['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']
columns_ratings = ['tconstIdentifier', 'averageRating', 'numVotes']

CLEAN_DATA = False # True to clean again the data, False to use the already pickled data

In [18]:
if CLEAN_DATA:
    #Load title_basics
    title_basics = load_metadata(TITLE_BASICS_DATASET, column_names=columns_title_basics)
    print("length of title_basics: ", len(title_basics))
    title_basics.head()

C:\Users\Alice\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


length of title_basics:  9363391


In [20]:
if CLEAN_DATA:
    #Load title_ratings
    ratings = load_metadata(TITLE_RATINGS_DATASET, column_names=columns_ratings)
    print("length of ratings: ", len(ratings))
    ratings.head()

length of ratings:  1246149


C:\Users\Alice\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Cleaning the dataset

In [22]:
if CLEAN_DATA:
    #Create a new table with only titleType=movies (get rid of videos, tvshows, tvepisodes and short)
    title_basics_movies = title_basics[title_basics["titleType"] == "movie"]
    #Remove the endYear column since movies are not concerned by thats
    title_basics_movies_cleaned = title_basics_movies.drop(columns='endYear')
    title_basics_movies_cleaned.replace('\\N',np.NaN,inplace=True) # replace \N by NaN
    # datetime format for dates
    title_basics_movies_cleaned.startYear = pd.to_datetime(title_basics_movies_cleaned.startYear,format='%Y').dt.year 
    title_basics_movies_cleaned.head()

## Saving the cleaned dataset

In [23]:
if CLEAN_DATA:
    #Pickle the data
    to_pickle_data = title_basics_movies_cleaned
    to_pickle_name = 'IMDb_title_movies'
    to_pickle_data.to_pickle(DESTINATION+to_pickle_name+EXT)

if not CLEAN_DATA: # for testing part
    # load already pickled data
    title_basics_movies_cleaned = pd.read_pickle("./Data/IMDb_title_movies.pkl")
    title_basics_movies_cleaned.startYear = pd.to_datetime(title_basics_movies_cleaned.startYear,format='%Y').dt.year

## Merging IMDb and CMU datasets

We match the movies from one dataset to the films on the other dataset on the movie name, as the ids are different.

In order to avoid mismatched pairs due to a little variation in the titles, we matched films of the same year, with almost identical titles. We create a dictionnary that matches the index of matched films.

In [24]:
copy_IMDb = title_basics_movies_cleaned.copy()
copy_IMDb = copy_IMDb[copy_IMDb.startYear >= 1910]
copy_CMU = movies.copy()
copy_CMU.dropna(subset=['Movie_box_office_revenue', 'Movie_release_date'], inplace=True)
copy_IMDb.dropna(subset= ['startYear'], inplace=True)

In [25]:
print(len(copy_CMU))
print(len(copy_IMDb))

8328
541993


In [26]:
import re
common_words = {'a','an','and','the','of','at','in'}
punctuation = {'.',',','!',';','?',''}
def compare(df1,df2,col1_title,col2_title,col1_year,col2_year,threshold = 0.8, delta_year=1):
    matched = {}
    count = 0
    for idx1,row1 in df1.iterrows():
        title1 = set(re.split('[ :,]',row1[col1_title].lower()))
        title1 = title1.difference(punctuation)
        y1 = row1[col1_year]
        #for idx2,row2 in df2[df2[col2_year].isin([y1-delta_year+i for i in range(delta_year*2)])].iterrows():
        for idx2,row2 in df2[df2[col2_year]==y1].iterrows():
            title2 = set(re.split('[ :,]',row2[col2_title].lower()))
            title2 = title2.difference(punctuation)
            if len(title1 & title2)/(len(title1 | title2)) > threshold:
                try:
                    matched[idx1].append(idx2)
                except KeyError:
                    matched[idx1] = [idx2]
        count += 1
        if count == 100: # remove for the whole computation
            break
    return matched

In [27]:
from time import time
deb = time()
matched = compare(copy_CMU,copy_IMDb, 'Movie_name', 'primaryTitle', 'Movie_release_date', 'startYear')
end = time()
print('Time of execution:', end-deb)
matched

Time of execution: 71.45266842842102


{0: [218707],
 7: [29325],
 13: [95308],
 17: [57208],
 29: [244954],
 36: [32552],
 49: [79055],
 53: [388561],
 54: [95478],
 61: [95595],
 85: [13738],
 88: [49725],
 99: [61508],
 119: [460855],
 124: [3795944],
 126: [62367],
 131: [77543],
 132: [45916],
 140: [112030],
 142: [75718],
 151: [85085],
 164: [116042],
 174: [450732],
 175: [66426],
 197: [358897],
 201: [29395],
 202: [75978],
 222: [399377],
 233: [76270],
 238: [105516],
 251: [838312],
 261: [76672],
 281: [117496],
 289: [100197],
 298: [391332],
 305: [101602],
 306: [116950],
 310: [85360],
 318: [138155],
 321: [99115],
 326: [299976],
 346: [117868],
 357: [108120],
 362: [44117],
 367: [91899],
 370: [448374],
 391: [4720360],
 431: [113744],
 464: [3200213],
 473: [158619],
 487: [105431],
 498: [27711],
 523: [765915],
 536: [353187],
 546: [353226],
 549: [110870],
 551: [3391684, 4535866],
 555: [1006903],
 562: [418100],
 575: [107409],
 576: [428090],
 595: [775146],
 599: [116360],
 605: [42029],
 60

In [28]:
# save the matching table
with open(DATA_FOLDER + 'matching_table.pkl', 'wb') as file:
        pickle.dump(matched, file, protocol=pickle.HIGHEST_PROTOCOL)